# Env setup

In [ ]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("Github")
# secret_value_1 = user_secrets.get_secret("wandb_key")

In [ ]:
# import wandb
# wandb.login(key=secret_value_1)

# From kagle dataaset

In [ ]:
import sys
sys.path.append('/kaggle/input/birdclef2025-code/main_folder/')
sys.path.append('/kaggle/input/birdclef2025-env/kaggle_env/')

In [ ]:
# test
from src.datasets.audio_dataset import AudioDataset

# Config manager

In [ ]:
# # copy config to working
# import shutil
# shutil.copytree("/kaggle/input/birdclef2025-code/main_folder/src/config", "/kaggle/working/config")

In [ ]:
# from omegaconf import OmegaConf

# cfg = OmegaConf.load("/kaggle/working/config/config.yaml") 
# print(OmegaConf.to_container(cfg, resolve=False))

In [ ]:
from hydra import initialize, compose
from omegaconf import OmegaConf
## "../input/birdclef2025-code/main_folder/src/config" - if from input

# if from workign after copy
with initialize(config_path="../input/birdclef2025-code/main_folder/src/config"): 
    cfg = compose(config_name="config")  # Load main config.yaml

# print(OmegaConf.to_container(cfg, resolve=False))

In [ ]:
cfg['inference']

In [ ]:
# paths
cfg['data']["paths"]['train_csv'] = "/kaggle/input/birdclef2025-cv-split/cv_split.csv"
cfg['inference']['train_audio_path'] = "/kaggle/input/birdclef-2025/train_audio"
cfg['inference']['test_soundscape_path'] ="/kaggle/input/birdclef-2025/test_soundscapes"
cfg['inference']['checkpoint_path'] = "/kaggle/input/birdclef2025-models/efficientnet_b0__weights_power-0.5-1st.ckpt"
cfg['inference']['batch_size'] = 2048

In [ ]:
# # test on train soundscapes
# cfg['inference']['test_soundscape_path'] ="/kaggle/input/birdclef-2025/train_soundscapes"
# cfg['inference']['test_soundscape_path']

# Inference

In [ ]:
from src.scripts.inference import main
main(cfg)

# Debug

In [ ]:
# import os
# import torch
# import librosa
# import numpy as np
# import pandas as pd
# from typing import List, Optional, Tuple
# from pathlib import Path
# from tqdm import tqdm
# from itertools import chain

# from src.models.spec_cnn import SpecCNNClassifier


# class Inference:
#     def __init__(
#         self,
#         model_path: str,
#         class_labels: List[str],
#         sample_rate: int = 32000,
#         target_duration: float = 5.0,
#         device: str = "cuda",
#         batch_size: int = 128,
#         model_config: Optional[dict] = None,
#     ):
#         self.model_path = model_path
#         self.class_labels = class_labels
#         self.sample_rate = sample_rate
#         self.target_duration = target_duration
#         self.device = device
#         self.batch_size = batch_size
#         self.model_config = model_config or {}
        
#         self.model = self._load_model()
#         self.model.eval()

#         self.logger = []
        
#     def _load_model(self) -> SpecCNNClassifier:
#         """Load the trained model from checkpoint."""
#         checkpoint = torch.load(self.model_path, map_location=self.device)
        
#         self.model_config['pretrained'] = False
#         model = SpecCNNClassifier(
#             **self.model_config,
#             device=self.device
#         )
        
#         if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
#             state_dict = checkpoint['state_dict']
#             state_dict = {k.replace('model.', ''): v for k, v in state_dict.items()}
#             model.load_state_dict(state_dict)
#         else:
#             model.load_state_dict(checkpoint)
            
#         model = model.to(self.device)
#         return model
    
#     def _prepare_audio_chunk(self, audio: np.ndarray) -> torch.Tensor:
#         """Prepare audio chunk for inference."""
#         target_length = int(self.sample_rate * self.target_duration)
#         if len(audio) < target_length:
#             audio = np.pad(audio, (0, target_length - len(audio)), mode='constant')
#         elif len(audio) > target_length:
#             start_idx = max(0, int(len(audio) / 2 - target_length / 2))
#             audio = audio[start_idx:start_idx + target_length]
            
#         audio = torch.from_numpy(audio).float()
#         return audio
    
#     def _get_audio_chunks(self, audio_path: str):
#         """Get all chunks from an audio file with their metadata."""
#         try:
#             audio, _ = librosa.load(audio_path, sr=self.sample_rate)
#             soundscape_id = Path(audio_path).stem
            
#             chunk_size = int(self.sample_rate * self.target_duration)
#             for i in range(0, len(audio), chunk_size):
#                 chunk = audio[i:i + chunk_size]
#                 yield (chunk, soundscape_id, i // chunk_size)
#         except Exception as e:
#             print(f"Error processing {audio_path}: {e}")
#             return []
    
#     def _process_batch(self, batch: List[Tuple[np.ndarray, str, int]]) -> pd.DataFrame:
#         """Process a batch of audio chunks and return predictions."""
#         audio_chunks, soundscape_ids, chunk_indices = zip(*batch)

#         # print(f"Soundscape ID, Chunk index = {list(zip(soundscape_ids, chunk_indices))}")
#         self.logger.append(list(zip(soundscape_ids, chunk_indices)))
        
#         batch_tensor = torch.stack([self._prepare_audio_chunk(chunk) for chunk in audio_chunks])
#         batch_tensor = batch_tensor.to(self.device)
        
#         with torch.no_grad():
#             output = self.model(batch_tensor)
#             logits = output["logits"]
#             scores = torch.sigmoid(logits).cpu().numpy()
        
#         predictions = pd.DataFrame(columns=['row_id'] + self.class_labels)
        
#         for i, (soundscape_id, chunk_idx) in enumerate(zip(soundscape_ids, chunk_indices)):
#             row_id = f"{soundscape_id}_{(chunk_idx + 1) * int(self.target_duration)}"
#             new_row = pd.DataFrame([[row_id] + list(scores[i])], 
#                                  columns=['row_id'] + self.class_labels)
#             predictions = pd.concat([predictions, new_row], axis=0, ignore_index=True)
            
#         return predictions
    
#     def predict_directory(self, directory_path: str) -> pd.DataFrame:
#         """Make predictions for all audio files in a directory."""
#         all_predictions = pd.DataFrame(columns=['row_id'] + self.class_labels)
        
#         audio_files = [os.path.join(directory_path, afile) 
#                             for afile in sorted(os.listdir(directory_path)) 
#                             if afile.endswith('.ogg')]
#         print(audio_files)

#         # test
#         audio_files = audio_files[:10]

#         batch = []
        
#         for audio_file in tqdm(audio_files, desc="Loading audio files"):
            
#             for chunk in self._get_audio_chunks(audio_file):
#                 batch.append(chunk)

#                 if len(batch) == self.batch_size:
#                     preds = self._process_batch(batch)

#                     all_predictions = pd.concat([all_predictions, preds], 
#                                       axis=0, ignore_index=True)
#                     batch = []
        
#         # finish remaining batches
#         if batch:
#             preds = self._process_batch(batch)
#             all_predictions = pd.concat([all_predictions, preds], 
#                                       axis=0, ignore_index=True)
        
#         for i in self.logger:
#             print(i)
#             print()

#         return all_predictions 

In [ ]:
# import os
# import hydra
# from omegaconf import DictConfig
# import pandas as pd
# from pathlib import Path


# model_path = Path(cfg.inference.checkpoint_path)
# test_soundscape_path = Path(cfg.inference.test_soundscape_path)


# df = pd.read_csv(cfg.data.paths.train_csv)
# class_labels = sorted(df.primary_label.unique())
# #sorted(os.listdir(cfg.data.train_audio_path))

# print(f"LEN LABELS:{len(class_labels)}")

# model_config = {
#     "backbone": cfg.model.backbone,
#     "n_classes": cfg.model.n_classes,
#     "classifier_dropout": cfg.model.classifier_dropout,
#     "top_db": cfg.model.top_db,
#     "spec_params": dict(cfg.model.spec_params),
#     "normalize_config": dict(cfg.model.normalize_config),
#     "pretrained": cfg.model.pretrained,
# }

# inference = Inference(
#     model_path=str(model_path),
#     class_labels=class_labels,
#     sample_rate=cfg.data.dataset_args.train_args.sample_rate,
#     target_duration=cfg.data.dataset_args.train_args.target_duration,
#     device=cfg.model.device,
#     batch_size = 128,
#     model_config=model_config,
# )

# predictions = inference.predict_directory(str(test_soundscape_path))


# os.makedirs(cfg.output_dir, exist_ok=True)
# output_path = Path(cfg.output_dir) / "submission.csv"

# predictions.to_csv(output_path, index=False)
# print(f"Predictions saved to {output_path}")
# print("\nFirst few predictions:")
# print(predictions.head())
